In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats, linalg
import time

import torch

In [2]:
TDC_Visa_ABC = pd.read_csv('TDC_Visa_Segmentado_ABC.csv')
TDC_Visa_ABC

,Socio,Mob,Saldo_total,Saldo_Mes,Pago_minimo_M0,Utilizacion,Pago_M0,Limite_credito,Genero,ESTADO,...,Edad,Variable_Objetivo_m6,Variable_Objetivo_m5,Variable_Objetivo_m4,Variable_Objetivo_m3,Variable_Objetivo_m2,Variable_Objetivo_m1,L2_Saldo_Mes,L2_Pago,Segmento_ABC
0,BRA,151,85488.45,85488.45,9014.00,0.949872,9050.0,90000.0,M,JAL,...,37,0,0,0,0,0,0,223586.542993,21500.000000,A
1,SUB,95,75568.64,75568.64,15869.42,1.109509,NaN,68110.0,M,TAM,...,30,0,0,0,0,0,0,166124.623360,19859.002996,A
2,SUB,97,90206.24,90206.24,90206.24,1.363662,NaN,66150.0,F,JAL,...,52,0,0,0,0,0,0,164640.000728,7522.196887,A
3,SUB,110,80815.94,80815.94,16782.37,1.221707,NaN,66150.0,F,JAL,...,67,0,0,0,0,0,0,161650.675505,14064.422811,A
4,CYA,63,72296.13,72296.13,72296.13,2.891845,NaN,25000.0,F,EM,...,50,0,0,0,0,0,0,159365.822611,9457.504956,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
853994,BOD,28,2.36,2.36,2.36,0.000615,236.0,3840.0,F,NL,...,66,0,0,0,0,0,0,2.360000,0.000000,C
853995,CYA,44,1.04,1.04,1.04,0.000217,2.0,4800.0,M,GTO,...,34,0,0,0,0,0,0,1.040000,0.000000,C
853996,BOD,11,0.84,0.84,0.84,0.000262,NaN,3200.0,F,QR,...,30,0,0,0,0,0,0,0.840000,0.000000,C
853997,BOD,15,0.27,0.27,0.27,0.000063,1.0,4300.0,F,GTO,...,33,0,0,0,0,0,0,0.270000,0.000000,C


In [3]:
from sklearn.model_selection import train_test_split

X = TDC_Visa_ABC.drop('Segmento_ABC', axis=1)  # features
y = TDC_Visa_ABC['Segmento_ABC']               # variable estratificadora

X_subset, _, y_subset, _ = train_test_split(
    X, y,
    test_size=0.985,      # mantener 30%
    stratify=y,         # estratificar por PP_ABC
    random_state=42
)

In [4]:
TDC_Visa_ABC = pd.concat([X_subset, y_subset], axis=1)
TDC_Visa_ABC.to_csv('TDC_Visa_ABC_ss.csv', index=False)
TDC_Visa_ABC

,Socio,Mob,Saldo_total,Saldo_Mes,Pago_minimo_M0,Utilizacion,Pago_M0,Limite_credito,Genero,ESTADO,...,Edad,Variable_Objetivo_m6,Variable_Objetivo_m5,Variable_Objetivo_m4,Variable_Objetivo_m3,Variable_Objetivo_m2,Variable_Objetivo_m1,L2_Saldo_Mes,L2_Pago,Segmento_ABC
562565,BOD,34,2850.41,2850.41,1659.05,2.591282,2850.41,1100.0,M,PUE,...,32,0,0,0,0,0,0,4560.744736,1199.490109,B
690775,BOD,3,2652.48,2652.48,411.05,1.153252,700.00,2300.0,F,GRO,...,40,0,0,0,0,0,0,3005.061845,1900.000000,C
630104,CYA,29,1953.45,1953.45,1206.00,1.415543,NaN,1380.0,F,JAL,...,36,0,0,0,0,1,1,3742.350273,3161.306692,B
449872,CYA,10,4405.70,4150.82,2711.00,0.620521,3000.00,7100.0,F,GTO,...,46,0,0,0,0,0,0,6034.061100,4527.692569,B
258736,PRO,7,7307.53,7307.53,1526.21,1.127705,7308.00,6480.0,M,DF,...,48,0,0,0,0,0,0,9999.464384,6105.597022,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
673066,BOD,3,2412.25,2412.25,1376.89,1.048804,2412.25,2300.0,F,MIC,...,40,0,0,0,0,0,0,3228.937968,1393.908481,C
362288,BOD,12,4768.73,4768.73,1879.02,1.018959,1000.00,4680.0,F,QR,...,34,0,0,0,0,0,0,7440.733347,644.127317,A
500007,BOD,27,2869.98,2869.98,966.48,1.434990,1000.00,2000.0,F,GRO,...,65,0,0,0,0,0,0,5371.565006,1950.119359,B
574270,SHA,30,3047.73,3047.73,1659.95,2.208500,NaN,1380.0,F,HGO,...,44,0,0,0,0,0,0,4409.272947,3355.964839,B
